In [1]:
#%###################################
#%#           Parity trees          #
#%###################################
#%ParityTree(num_of_sites,Parents)
struct PTree
    M::Int64
    Parents::Array{Int64}
end

#-1 indicates the node has no parent
function JWTree(M)
    Parents=-ones(M,1)
    return PTree(M,Parents)
end


#%Parity tree functions
#%--
#%function Ancestors(      j,PT::PTree)
#%function Children(       j,PT::PTree)
#%function YoungerCousins( j,PT::PTree)
#%function DisjointRoots(  j,PT::PTree)
#%function Progeny(        j,PT::PTree)

JWTree (generic function with 1 method)

In [8]:
function Ancestors(j,PT::PTree,verbose::Bool=false)
    A=[];
    cur=j;
    ctr=1;
    while ctr<1000
        if(verbose)
            println("ctr - ",ctr)
        end
        if(PT.Parents[cur]==-1)
            if(verbose)
                println("breaking val of cur=",cur)
            end
            break;
        else
            A=[A; PT.Parents[cur]];
            cur=PT.Parents[cur];
        end
        
        ctr=ctr+1
    
    end
    if ctr>PT.M
        if(verbose)
            println("ctr - ",ctr,"; PT.M - ",PT.M)
        end
        print("Warning, the loop didn't close automatically")
    end
    
    return A;
end



Ancestors (generic function with 2 methods)

In [22]:
pt=ParityTree(3)
Ancestors(1,pt)

2-element Array{Any,1}:
 2
 3

In [23]:
jw=JWTree(3)
Ancestors(1,jw)

0-element Array{Any,1}

In [24]:
function ParityTree(N)

  paritytree=-ones(N,1);

  for i=1:(N-1)
    paritytree[i]=i+1;
  end

  return PTree(N,paritytree);
end


ParityTree (generic function with 1 method)

In [25]:

function Children(j,PT::PTree)
    C=[];
    for k=1:PT.M
        if(PT.Parents[k]==j)
            C=[C; k];
        end
    end
    return C;
end



Children (generic function with 1 method)

In [26]:
Children(3,pt)

1-element Array{Any,1}:
 2

In [27]:
function Progeny(j,tree::PTree)
  #children's children
  all_kids=[];

  ctr=0;
  g0kids=Children(j,tree);


  while(length(g0kids)>0)
    #Basis cases to illustrate the logic
    #
    # kids=Children(j)
    # grandkids=[];
    # for i=1:length(kids)
    #   child=kids[i];
    #   grandkids=[grandkids Children(child)];
    # end
    #
    # greatgrandkids=[];
    # for i=1:length(grandkids)
    #   grandchild=grandkids[i];
    #   greatgrandkids=[greatgrandkids Children(grandchild)];
    # end

    #g0 is the previous generation, g1 is the next generation
    g1kids=[];
    for i=1:length(g0kids)

      #pull child
      child0=g0kids[i];

      g1kids=append!(g1kids,Children(child0,tree));
    end

    #record this generation
    all_kids=append!(all_kids,g0kids);

    #check the next generation
    g0kids=g1kids;

    ctr=ctr+1;
    #prevent excessive nesting
    if(ctr>100)
      print("\n\tWarning: Excessive nesting, pruning\n")
      return;
    end
  end

  return all_kids;
end



Progeny (generic function with 1 method)

In [28]:
Progeny(3,pt)

2-element Array{Any,1}:
 2
 1

In [29]:

function YoungerCousins(j,PT::PTree)
    K=[];
    A=Ancestors(j,PT)
    for k=1:length(A)
        Ck=Children(A[k],PT);
        for kk=1:length(Ck)
            if(Ck[kk]<j)
                K=[K; Ck[kk]]
            end
        end
    end
    return K;
end



YoungerCousins (generic function with 1 method)

In [30]:
YoungerCousins(3,jw)

0-element Array{Any,1}

In [31]:

function DisjointRoots(j,PT::PTree)
    L=[];

    for k=j-1:-1:1
        if PT.Parents[k]==-1;
            L=[L; k];
        end
    end

    return L
end



DisjointRoots (generic function with 1 method)

In [32]:
DisjointRoots(3,jw)

2-element Array{Any,1}:
 2
 1

In [33]:
DisjointRoots(2,pt)

0-element Array{Any,1}

In [34]:
function ShiftTreeIndices(shift,parent_tree)
  shifted_tree=parent_tree.+shift;

  #if it used to be empty it needs to reset
  shifted_tree[shifted_tree .== shift-1].=-1;
  return shifted_tree
end



ShiftTreeIndices (generic function with 1 method)

In [35]:
ShiftTreeIndices(4,pt.Parents)

3×1 Array{Int64,2}:
  6
  7
 -1

In [36]:
st=pt.Parents.+1
idx=st.==0
st[3]=-1
#st[idx]=-1

-1

In [37]:
st[st.==-1].=0

1-element SubArray{Int64,1,Array{Int64,1},Tuple{Array{Int64,1}},false}:
 0

In [38]:
st

3×1 Array{Int64,2}:
 3
 4
 0

In [39]:
pt

PTree(3, [2; 3; -1])

In [40]:
function CombineTrees(treeL,treeR)
  m=length(treeL)
  shifted_treeR=ShiftTreeIndices(m,treeR)
  return [treeL;shifted_treeR]
end



CombineTrees (generic function with 1 method)

In [41]:
ct=CombineTrees(pt.Parents,pt.Parents)

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [42]:
ct==[2;3;-1;5;6;-1]

false

In [43]:
Array(ct)

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [44]:
Array([2;3;-1;5;6;-1])≈ Array(ct)

true

In [45]:
ct[2]

3

In [46]:
ct

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [47]:
ct[3]

-1

In [48]:
ct[4]

5

In [49]:
function BetaMatrix(PT::PTree)

  #make MxM zeros matrix
  B=zeros(Int64,PT.M,PT.M)

  #for each node in tree
  for j=1:PT.M

    #get progeny list
    pj=Progeny(j,PT)

    #mark appropriate matrix elements

    #use the state ordering from eq (24) of Seeley12

    #equation 24 of peter's paper has a strange ordering
    #based on b_i=\sum B_{ij} f_j instead of the usual change of basis formula
    # b_i=\sum B_{ji} f_j

    B[j,j]=1;
    for k=1:length(pj)
      B[PT.M+1-j,PT.M+1-pj[k]]=1;
    end

  end

  #return matrix
  return B;
end

BetaMatrix (generic function with 1 method)

In [50]:
BetaMatrix(jw)

3×3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1

In [51]:

#%######################################
#% Fenwick tree and related structures #
#%######################################
#%--
#% tree = MakeFenwickTree(M)
#% PT = JWTree(M)
#% PT = BKTree(M)
#% PT = SBKTree(M,stride_length)
#% PT = ParityTree(M)

function MakeFenwickTree(N)

    if(N==0)
      return [];
    end
    #global variables with respect to FENWICK
    tree=-ones(N,1)
    ctr=0;

    function FENWICK(L,R)
        #The recursive algorithm Vojta suggested in our paper


        if(ctr>100)
            #prevent excessive nesting
            print("\n\tWarning: Excessive nesting, pruning\n")
            return;
        end

        #algorithm is to connect from middle between L & R to right then recuse
        #on each half. Be careful about floor versus ceiling.

        if(L==R)            #if L=R then there is no middle, so we're done
            return;
        end
        n=Int64(floor((L+R)/2))

        #Connect middle to R
        #+1 because we count from 1 in this language (Julia)
        tree[n+1]=R+1;

        #branch
        ctr=ctr+1;
        FENWICK(L+0.0, n)
        FENWICK(1.0+n, R)

    end

    #run the recursive algorithm on (0,N-1) and it generates the BK tree over N
    #nodes.
    FENWICK(0.0,N-1.0)
    return tree
end

function JWTree(M)
    Parents=-ones(M,1)
    return PTree(M,Parents)
end

function BKTree(M)
  return PTree(M,MakeFenwickTree(M))
end

function SBKTree(M,L)
  #L=stride_length;

  #how many strides?
  n=M/L;
  n=n-n%1;
  #remainder
  r=M-n*L;

  sktree=[];
  #Now we just need to make n strides
  for i=1:n
    sktree=CombineTrees(MakeFenwickTree(L),sktree);
  end
  #plus the remaining nodes

  sktree=CombineTrees(MakeFenwickTree(r),sktree);

  return PTree(M,sktree);
end

function ParityTree(M)

  paritytree=-ones(M,1);

  for i=1:(M-1)
    paritytree[i]=i+1;
  end

  return PTree(M,paritytree);
end



ParityTree (generic function with 1 method)

In [52]:
bk=BKTree(4)

PTree(4, [2; 4; 4; -1])

In [54]:
function MultiplyPauli(A,B)
  algebra=GetSpinAlgebra();
  return algebra[A,B];
end

MultiplyPauli (generic function with 1 method)

In [55]:
MultiplyPauli(Z,X)

0.0f0 + 3.0f0im

In [56]:
macro isdefined(var)
                  quote
                      try
                          local _= $( esc(var) )
                          true
                      catch err
                          isa(err, UndefVarError) ? false : rethrow(err)
                      end#try
                  end#quote
              end#macro##

@isdefined (macro with 1 method)

In [93]:
include("Spin_Algebra.jl")

LoadError: LoadError: [91minvalid redefinition of constant PauliList[39m
while loading /Volumes/JDWHITFIELD/files/opt/Julia-qFenwick/Spin_Algebra.jl, in expression starting on line 218

In [70]:
#basic definitions
function c_op(j,PT::PTree)
  # c_j = Z_{DisjointRoots} Z_{Children} Z_{YCousins} X_j X_{Ancestors}

  #These are the MatrixIDs for coding the algebra
  I   =1;
  X   =2;
  Y   =3;
  Z   =4;
  P00 =5;
  P01 =6;
  P10 =7;
  P11 =8;
  ops=ones(Complex,PT.M);

  #Z_{DisjointRoots}
  roots=DisjointRoots(j,PT)
  ops[roots]=Z;

  #Z_children
  children=Children(j,PT)
  ops[children]=Z;

  #Z_cousins
  cousins=YoungerCousins(j,PT)
  ops[cousins]=Z;

  #X_j
  ops[j]=X;

  #X_ancestors
  elders=Ancestors(j,PT)
  ops[elders]=X;

  return PauliList(1,ops)
end

c_op (generic function with 1 method)

In [71]:
Print(c_op(2,jw))

LoadError: [91mtype PauliList is immutable[39m

In [73]:
ZZ=PauliList(1,[Z,Z])

PauliList(1 + 0im, Complex[4+0im, 4+0im])

In [74]:
ZZ.List

2-element Array{Complex,1}:
 4+0im
 4+0im

In [79]:
sum(abs.(ZZ.List).<1.0)

0

In [80]:
b=(ZZ.List)./abs.(ZZ.List)

2-element Array{Complex{Float64},1}:
 1.0+0.0im
 1.0+0.0im

In [85]:
prod(b[.!isnan.(b)])

1.0 + 0.0im

In [91]:
Complex.(round.(float(abs.(ZZ.List))))

2-element Array{Complex{Float64},1}:
 4.0+0.0im
 4.0+0.0im